<a href="https://colab.research.google.com/github/tupiribas/analise-dados-bcb/blob/main/Analise_de_IPCA_por_Setor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from os import system, name
from sys import modules



def limpar_tela() -> None:
  '''Limpar tela do terminal'''
  # Verifica se está executando no google colab ou no computador
  if 'google.colab' in modules:
    from google.colab import output
    output.clear()
  elif name == 'nt':
    system('cls')
  else:
    system('clear')

def input_com_validacao(texto, tipo: type[str|int|float]):
  '''Validar o input e verificar se o tipo de dado é correto com o limite de tentativas'''
  while True:
    dados = input(texto)
    try:
      return tipo(dados) # Processamento/Validação no input
    except ValueError:
      print("Entrada inválida. Tente novamente.")

def validacao_tipo_indice(tipo_indice:str) -> str:
  '''Retorna a URL do índice financeiro ou levanta um erro se for inválido.'''

  # Limpa o texto evitando problemas de espaço
  indice_limpo = tipo_indice.replace('-', '').strip().lower()

  urls = {
      'igpm': 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.189/dados?formato=json',
      'ipca': 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.443/dados?formato=json',
      'incc': 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.7456/dados?formato=json'
  }

  if indice_limpo in urls:
    return urls[indice_limpo]
  else:
    raise ValueError(f'Indice {indice_limpo} inválido. Escolha apenas IPCA, IGP-M ou INCC')


try:
  # Obter o indice
  tipo_indice = input('Qual é o indexador desse contrato (IPCA|IGP-M|INCC)?')

  # Obter a quantidade em anos
  qtde_anos = input_com_validacao('Digite a quantidade de anos?', int)

  # Obter o api correspondente a opção selecionada
  url_bcb = validacao_tipo_indice(tipo_indice)

  # Consumo dos dados da api
  df_ipca = pd.read_json(
      url_bcb,
      encoding='utf-8'
  )

  # Trasformar a coluna data em datetime
  df_ipca['data'] = pd.to_datetime(df_ipca['data'], format="%d/%m/%Y")
  if df_ipca['valor'].isna().sum():
    print('Tem dados nulos nos dados fornecidos pelo banco central!\n Tente mais tarde.')
    raise

  # Cria uma coluna ano obtendo apenas o ano do datetime
  df_ipca['ano'] = df_ipca['data'].dt.year

  # Obter o ano base e atual
  ano_atual = df_ipca['ano'].max()
  ano_base = ano_atual - qtde_anos

  # Dividir a coluna valor por 100 e somar por 1
  df_ipca['fator'] = 1 + (df_ipca['valor'] / 100)

  # Filtrar pelo período
  df_recentes = df_ipca[df_ipca['ano'] >= ano_base]

  # Agrupar e soma pelo período mais recente
  result_prod = df_recentes.groupby('ano')['fator'].prod() - 1

  # Percentual do resultando do produto
  percentual = result_prod * 100

  # Apresentação dos resultado do somatório
  print(f'A inflação anual é essa ({ano_base}-{ano_atual}): {percentual}\n', percentual.plot(kind='bar'))

except ConnectionError as e:
  print("❌ O servidor do Banco Central pode estar instável ou fora do ar, isso é um problema de disponibilidade da informação do banco central \nINFO ERRO:\n", e)
except ValueError as e:
  print('Algum valor está inadequado ou inapropriado para a operação, isso é com o desenvolvedor do sistema:\nINFO ERRO:\n', e)
except AttributeError as e:
  print('Alguns valores ainda se encontram com string/object, isso é com o desenvolvedor: \nINFO ERRO:\n', e)
except Exception as e:
  print('Erro excepcional: \nINFO ERRO:\n', e)
except KeyboardInterrupt:
  print('Programa foi interrompido inesperadamente!')

Qual é o indexador desse contrato (IPCA|IGP-M|INCC)?cu
Digite a quantidade de anos?5
Algum valor está inadequado ou inapropriado para a operação, isso é com o desenvolvedor do sistema:
INFO ERRO:
 Indice cu inválido. Escolha apenas IPCA, IGP-M ou INCC
